In [95]:
from netCDF4 import Dataset
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import xarray as xr
import pdb
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [71]:
def pre_proc(mdir,run,time):
    ds_tem = xr.open_dataset(mdir+run+'_temp_1000-875.nc')
    ds_qv = xr.open_dataset(mdir+run+'_qv_1000-875.nc')
    ds_ql = xr.open_dataset(mdir+run+'_ql_1000-875.nc')
    lista = [ds_tem,ds_qv,ds_ql]
    varis = ['__xarray_dataarray_variable__','QVAPOR','QCLOUD']
    dfs = []
    for i,li in enumerate(lista):
        vari = li[varis[i]][time,:,:,:].stack(space=['south_north','west_east'])
        vari = np.array(vari).flatten()
        dfs.append(pd.DataFrame(vari))
    return(dfs)
    
def Data_frame(ds):
    df = pd.DataFrame.from_records(np.array(ds))
    return(df)

In [72]:
mdir = '/Users/Casi94/Documents/PhD/Thesis/Idealized/clouds/'

run = 'ThoYSU'

dfs = pre_proc(mdir,run,20)

In [73]:
X = pd.concat([dfs[0].add_suffix('_temp'),dfs[1].add_suffix('_qv')], axis = 1)
y = dfs[2].add_suffix('_ql')

In [92]:
perc = 0.8
X_train = X[0:int(len(X)*perc)]
X_test = X[int(len(X)*perc)+1:]

y_train = y[0:int(len(y)*perc)]
y_test = y[int(len(y)*perc)+1:]

# fit the network
scaler = StandardScaler()
#X_trainscaled=sc_X.fit_transform(X_train)
#X_testscaled=sc_X.transform(X_test)

# do manual scaling as I need to know this for later.
xs=scaler.fit(X_train)
X_trainscaled=scaler.transform(X_train)
X_testscaled=scaler.transform(X_test)

In [98]:
nn=64

#reg2=MLPRegressor(hidden_layer_sizes=(nn,nn,nn,nn),activation="relu" ,random_state=1, max_iter=3000)
#reg2.fit(X_trainscaled, y_train)

yNN = RandomForestRegressor()
yNN.fit(X_trainscaled, y_train)

/Users/Casi94/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [99]:
y_NN=reg2.predict(X_testscaled)

r2skill=r2_score(y_test, y_NN)
bias=np.mean(y_NN)-np.mean(y_test)
rmse=np.sqrt(mean_squared_error(y_test, y_NN))
print("r2",r2skill,"bias",bias,"RMSE", rmse)

r2 -0.011027924909946973 bias -3.8089267e-06 RMSE 3.7278845e-05
